In [ ]:
#NYC final

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
# CASSANDRA CONFIGURATION
cassandra_host = "cassandra"
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.cassandra.connection.host", cassandra_host) \
      .config("spark.jars.packages","com.datastax.spark:spark-cassandra-connector-assembly_2.12:3.1.0")\
    .getOrCreate()
sc = spark.sparkContext

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.datastax.spark#spark-cassandra-connector-assembly_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-684a5492-3479-401a-9f4f-dc7099e48d34;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 in central
:: resolution report :: resolve 451ms :: artifacts dl 7ms
	:: modules in use:
	com.datastax.spark#spark-cassandra-connector-assembly_2.12;3.1.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: 

In [3]:
!pip install -q cassandra-driver
!pip install pandas
!pip install sodapy

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
!pip install pandas
!pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("erm2-nwe9", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [20]:

import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    
data_set='erm2-nwe9'    
app_token='T2Jeqm4J7zvAiJruefHcYvb7y' 
selectc = "unique_key, created_date, agency, complaint_type, descriptor, location_type, intersection_street_1, intersection_street_2 ,address_type, status,resolution_action_updated_date, community_board, borough, open_data_channel_type, incident_zip, incident_address, city, latitude, longitude"
client = Socrata(data_url,app_token)      
client.timeout = 360
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
results = client.get(data_set,select=selectc,  limit=2000)
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
# Save the data frame to a CSV/JSON file
df.to_json("my_311_data.json", orient="records")
df.to_csv("my_311_data.csv")


In [4]:

file_path = 'file:///home/jovyan/datasets/my_311_data.json'
df311= spark.read.json(file_path)
df311.printSchema()
#save as dataframe and see the schema 


root
 |-- address_type: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_board: string (nullable = true)
 |-- complaint_type: string (nullable = true)
 |-- created_date: string (nullable = true)
 |-- descriptor: string (nullable = true)
 |-- incident_address: string (nullable = true)
 |-- incident_zip: string (nullable = true)
 |-- intersection_street_1: string (nullable = true)
 |-- intersection_street_2: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- location_type: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- open_data_channel_type: string (nullable = true)
 |-- resolution_action_updated_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- unique_key: string (nullable = true)



In [13]:
df311.count()

2000

In [14]:
df311.select('complaint_type').distinct().count()

81

In [20]:
create_table_sql = '''
CREATE TABLE project311.data
(

 address_type text,
 agency text,
 borough text,
 city text,
 community_board text,
 complaint_type text,
 created_date timestamp,
 descriptor text,
 incident_address text,
 incident_zip text,
 intersection_street_1 text,
 intersection_street_2 text,
 latitude float,
 location_type text,
 longitude float,
 open_data_channel_type text,
 resolution_action_updated_date timestamp,
 status text,
 unique_key text,
 PRIMARY KEY (unique_key)
  
  );
  '''
from cassandra.cluster import Cluster
with Cluster([cassandra_host]) as cluster:
    session = cluster.connect()
    session.execute(create_table_sql)
print(create_table_sql)
        


CREATE TABLE project311.data
(

 address_type text,
 agency text,
 borough text,
 city text,
 community_board text,
 complaint_type text,
 created_date timestamp,
 descriptor text,
 incident_address text,
 incident_zip text,
 intersection_street_1 text,
 intersection_street_2 text,
 latitude float,
 location_type text,
 longitude float,
 open_data_channel_type text,
 resolution_action_updated_date timestamp,
 status text,
 unique_key text,
 PRIMARY KEY (unique_key)
  
  );
  


In [21]:
df2 = df311.toDF(



"address_type",
"agency",
"borough",
"city",
"community_board",
"complaint_type",
"created_date",
"descriptor",
"incident_address",
"incident_zip",
"intersection_street_1",
"intersection_street_2",
"latitude",
"location_type",
"longitude",
"open_data_channel_type",
"resolution_action_updated_date",
"status",
"unique_key")

In [22]:
df2.write.format("org.apache.spark.sql.cassandra")\
    .mode("Append")\
    .option("table", "data")\
    .option("keyspace", "project311")\
    .save()

In [5]:
df3 =spark.read.format("org.apache.spark.sql.cassandra")\
    .option("table", "data")\
    .option("keyspace","project311")\
    .load()

df3.count()

2000

In [6]:
df3.printSchema()

root
 |-- unique_key: string (nullable = false)
 |-- address_type: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- city: string (nullable = true)
 |-- community_board: string (nullable = true)
 |-- complaint_type: string (nullable = true)
 |-- created_date: timestamp (nullable = true)
 |-- descriptor: string (nullable = true)
 |-- incident_address: string (nullable = true)
 |-- incident_zip: string (nullable = true)
 |-- intersection_street_1: string (nullable = true)
 |-- intersection_street_2: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- location_type: string (nullable = true)
 |-- longitude: float (nullable = true)
 |-- open_data_channel_type: string (nullable = true)
 |-- resolution_action_updated_date: timestamp (nullable = true)
 |-- status: string (nullable = true)



Py4JJavaError: An error occurred while calling o55.save.
: java.lang.ClassNotFoundException: Failed to find data source: es. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:993)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:311)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:293)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j